In [1]:
from global_variables import *

{'command': {'check_sequence': 'code/check_sequence.sh 8 output '
                               'input/sample/*fastq*',
             'process_sample': 'python code/process_sample.py'},
 'download': {'input': ['data_for_sequencing_process.zip']},
 'environment': ['pip install kraft',
                 'conda install --prefix PROJECT_ENVIRONMENT_DIRECTORY_PATH '
                 '--channel bioconda --yes htslib',
                 'conda install --prefix PROJECT_ENVIRONMENT_DIRECTORY_PATH '
                 '--channel bioconda --yes samtools'],
 'gb_memory': 12,
 'germ_dna.1': '../input/sample/germ_dna.1.fastq.gz',
 'germ_dna.2': '../input/sample/germ_dna.2.fastq.gz',
 'git_url': 'https://github.com/Guardiome/sample_template',
 'keyword': [],
 'n_job': 8,
 'name': 'Example Sample',
 'soma_dna.1': '../input/sample/soma_dna.1.fastq.gz',
 'soma_dna.2': '../input/sample/soma_dna.2.fastq.gz',
 'soma_rna.1': '../input/sample/soma_rna.1.fastq.gz',
 'soma_rna.2': '../input/sample/soma_rna.2.fastq

In [2]:
from kraft import make_variant_n_from_vcf_file_path, run_command

In [3]:
if "germ_dna.1" in PROJECT_JSON and "germ_dna.2" in PROJECT_JSON:

    print("Processing germ DNA ...")

    run_command(
        "{}/process_germ_dna.sh {} {} {} {} {} exome {}".format(
            CODE_DIRECTORY_PATH,
            DATA_DIRECTORY_PATH,
            PROJECT_JSON["n_job"],
            PROJECT_JSON["gb_memory"],
            PROJECT_JSON["germ_dna.1"],
            PROJECT_JSON["germ_dna.2"],
            OUTPUT_GERM_DNA_DIRECTORY_PATH,
        )
    )

Processing germ DNA ...
/home/kwat/github/sample_template/code/process_germ_dna.sh /media/kwat/CarrotCake/data/ 8 12 ../input/sample/germ_dna.1.fastq.gz ../input/sample/germ_dna.2.fastq.gz exome /home/kwat/github/sample_template/output/germ_dna


In [4]:
if "soma_dna.1" in PROJECT_JSON and "soma_dna.2" in PROJECT_JSON:

    print("Processing soma DNA ...")

    run_command(
        "{}/process_soma_dna.sh {} {} {} {} {} {} {} exome {}".format(
            CODE_DIRECTORY_PATH,
            DATA_DIRECTORY_PATH,
            PROJECT_JSON["n_job"],
            PROJECT_JSON["gb_memory"],
            PROJECT_JSON["germ_dna.1"],
            PROJECT_JSON["germ_dna.2"],
            PROJECT_JSON["soma_dna.1"],
            PROJECT_JSON["soma_dna.2"],
            OUTPUT_SOMA_DNA_DIRECTORY_PATH,
        )
    )

Processing soma DNA ...
/home/kwat/github/sample_template/code/process_soma_dna.sh /media/kwat/CarrotCake/data/ 8 12 ../input/sample/germ_dna.1.fastq.gz ../input/sample/germ_dna.2.fastq.gz ../input/sample/soma_dna.1.fastq.gz ../input/sample/soma_dna.2.fastq.gz exome /home/kwat/github/sample_template/output/soma_dna


In [5]:
if "soma_rna.1" in PROJECT_JSON and "soma_rna.2" in PROJECT_JSON:

    print("Processing soma RNA ...")

    run_command(
        "{}/process_soma_rna.sh {} {} {} {} {} {}".format(
            CODE_DIRECTORY_PATH,
            DATA_DIRECTORY_PATH,
            PROJECT_JSON["n_job"],
            PROJECT_JSON["gb_memory"],
            PROJECT_JSON["soma_rna.1"],
            PROJECT_JSON["soma_rna.2"],
            OUTPUT_SOMA_RNA_DIRECTORY_PATH,
        )
    )

Processing soma RNA ...
/home/kwat/github/sample_template/code/process_soma_rna.sh /media/kwat/CarrotCake/data/ 8 12 ../input/sample/soma_rna.1.fastq.gz ../input/sample/soma_rna.2.fastq.gz /home/kwat/github/sample_template/output/soma_rna


In [6]:
from os import mkdir
from os.path import isdir, join
from shutil import rmtree

In [7]:
if isdir(SUMMARY_DIRECTORY_PATH):

    rmtree(SUMMARY_DIRECTORY_PATH)

mkdir(SUMMARY_DIRECTORY_PATH)

In [8]:
if isdir(OUTPUT_GERM_DNA_DIRECTORY_PATH):

    germ_variant_n = make_variant_n_from_vcf_file_path(
        join(OUTPUT_GERM_DNA_DIRECTORY_PATH, "snpeff", "variant.vcf.gz")
    )

    germ_variant_n.to_csv(
        join(SUMMARY_DIRECTORY_PATH, "germ_dna.variant_n.tsv"), sep="\t", header=True
    )

    print(germ_variant_n)

Using only rows with 'PASS' ...
Variant
NTRK3 (intron_variant)                                              202
HLA-DQB1 (upstream_gene_variant)                                    186
HLA-DQB1 (intron_variant)                                           183
BCR (intron_variant)                                                178
NRG1 (intron_variant)                                               176
FGFR2 (intron_variant)                                              168
HLA-DQB1-AS1 (downstream_gene_variant)                              141
HLA-DQB1 (non_coding_transcript_exon_variant)                       111
HLA-C (upstream_gene_variant)                                       100
HLA-DQA1 (intron_variant)                                            97
HLA-B (upstream_gene_variant)                                        90
HLA-DQA1 (downstream_gene_variant)                                   88
PAX8 (intron_variant)                                                88
HLA-C (non_coding_transc

In [9]:
if isdir(OUTPUT_SOMA_DNA_DIRECTORY_PATH):

    soma_variant_n = kraft.make_variant_n_from_vcf_file_path(
        join(OUTPUT_SOMA_DNA_DIRECTORY_PATH, "snpeff", "variant.vcf.gz")
    )

    soma_variant_n.to_csv(
        join(SUMMARY_DIRECTORY_PATH, "soma_dna.variant_n.tsv"), sep="\t", header=True
    )

    print(soma_variant_n)

NameError: name 'kraft' is not defined

In [ ]:
from pandas import Series, read_csv

In [ ]:
if isdir(OUTPUT_SOMA_RNA_DIRECTORY_PATH):

    enst_tpm = read_csv(
        join(
            OUTPUT_SOMA_RNA_DIRECTORY_PATH, "kallisto", "transcriptome", "abundance.tsv"
        ),
        sep="\t",
        index_col=0,
    )["tpm"].sort_values(ascending=False)

    enst_tpm.index.name = "ENST"

    enst_tpm.name = "TPM"

    enst_tpm.to_csv(join(SUMMARY_DIRECTORY_PATH, "enst_tpm.tsv"), sep="\t", header=True)

    print(enst_tpm)

    enst_gene_name = read_csv(
        join(DATA_DIRECTORY_PATH, "grch", "enst_gene_name.tsv"), sep="\t"
    )

    enst_gene_name = dict(
        zip(enst_gene_name["Transcript stable ID version"], enst_gene_name["Gene name"])
    )

    gene_tpm = Series(
        enst_tpm.values,
        index=enst_tpm.index.map(lambda enst: enst_gene_name.get(enst, enst)),
    )

    size_before = gene_tpm.size

    gene_tpm = gene_tpm.groupby(by=gene_tpm.index).median().sort_values(ascending=False)

    print("Size: {} =(groupby)=> {}".format(size_before, gene_tpm.size))

    gene_tpm.index.name = "Gene"

    gene_tpm.name = "Median TPM"

    gene_tpm.to_csv(join(SUMMARY_DIRECTORY_PATH, "gene_tpm.tsv"), sep="\t", header=True)

    print(gene_tpm)

    virus_id_tpm = read_csv(
        join(OUTPUT_SOMA_RNA_DIRECTORY_PATH, "kallisto", "virus", "abundance.tsv"),
        sep="\t",
        index_col=0,
    )["tpm"].sort_values(ascending=False)

    virus_id_tpm.index.name = "Virus ID"

    virus_id_tpm.name = "TPM"

    virus_id_tpm.to_csv(
        join(SUMMARY_DIRECTORY_PATH, "virus_id_tpm.tsv"), sep="\t", header=True
    )

    print(virus_id_tpm)

    virus_id_name = read_csv(
        join(DATA_DIRECTORY_PATH, "virus", "sequences.csv"), index_col=0
    )["Species"].to_dict()

    virus_name_tpm = Series(
        virus_id_tpm.values,
        index=virus_id_tpm.index.map(
            lambda virus_id: virus_id_name.get(virus_id, virus_id)
        ),
    )

    size_before = virus_name_tpm.size

    virus_name_tpm = (
        virus_name_tpm.groupby(by=virus_name_tpm.index)
        .max()
        .sort_values(ascending=False)
    )

    print("Size: {} =(groupby)=> {}".format(size_before, virus_name_tpm.size))

    virus_name_tpm.index.name = "Virus Name"

    virus_name_tpm.name = "Max TPM"

    virus_name_tpm.to_csv(
        join(SUMMARY_DIRECTORY_PATH, "virus_name_tpm.tsv"), sep="\t", header=True
    )

    print(virus_name_tpm)